In [15]:
%load_ext autoreload
%autoreload 2

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp
from pathlib import Path
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import sys
sys.path.append('../src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from extended_survey import process_people_df, process_places_df, categorize_p, categorize_v

# Define data paths
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')

# Load survey data
personas = process_people_df(personas_path)
viviendas = process_places_df(viviendas_path)

# Select subset of categorical columns to control for
# Seleting them before dropping nan from survey
# Alternative is to impute NAN (e.g. use missforest)
pcat = personas[[
    'ID_PERSONA', 'ID_VIV', 'FACTOR', 'MUN',
    'SEXO', 'EDAD',
    # 'ENT_PAIS_NAC',
    # 'AFRODES',
    'DHSERSAL1', 'DHSERSAL2', 'RELIGION',
    # 'DIS_VER', 'DIS_OIR', 'DIS_CAMINAR', 'DIS',
    # 'DIS_RECORDAR', 'DIS_BANARSE', 'DIS_HABLAR', 'DIS_MENTAL',
    # 'HLENGUA',
    # 'HESPANOL',  # Global seed zero problem 
    'ASISTEN', 'NIVACAD', 'ESCOLARI', 'ALFABET',
    # 'ENT_PAIS_RES_5A',
    'SITUA_CONYUGAL', 'CONACT',
    'INGTRMEN', 'HORTRA'
]].copy()
vcat = viviendas.copy()

# Drop NA values on both surveys

# Look for viviendas with NA values in people and household constraints
na_vivs_v = vcat.ID_VIV[vcat.isna().T.sum() > 0].to_list()
na_vivs_p = pcat.ID_VIV[pcat.isna().T.sum() > 0].to_list()
na_vivs = set(na_vivs_v + na_vivs_p)

# Drop NA before categorizing
pcat = pcat[~pcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)
vcat = vcat[~vcat.ID_VIV.isin(na_vivs)].reset_index(drop=True)

# Categorize columns
pcat = categorize_p(pcat)
vcat = categorize_v(vcat)

assert pcat.isna().sum().sum() == 0
assert vcat.isna().sum().sum() == 0

# Leave only categorized columns
pcat = pcat.drop(columns=[
    'DHSERSAL1', 'DHSERSAL2',
    'NIVACAD', 'ESCOLARI',
    'INGTRMEN', 'HORTRA',
    'DHSERSAL_IMSS', 'DHSERSAL_ISSSTE', 'DHSERSAL_ISSSTE_E',
    'DHSERSAL_P_D_M', 'DHSERSAL_Popular_NGenración_SBienestar',
    'DHSERSAL_IMSS_Prospera/Bienestar'
])

vcat = vcat[[
    'ID_VIV', 'FACTOR', 'MUN', 'NUMPERS',
    'CLAVIVP',
    # 'PISOS',
    'CUADORM', 'TOTCUART',
    # 'ELECTRICIDAD', 'AGUA_ENTUBADA',
    # 'ABA_AGUA_ENTU',
    # 'TINACO', 'CISTERNA',
    # 'SERSAN',
    # 'CONAGUA',
    # 'DRENAJE',
    'REFRIGERADOR', 'LAVADORA', 'HORNO',
    'AUTOPROP', 'MOTOCICLETA', 'BICICLETA', 'RADIO', 'TELEVISOR',
    'COMPUTADORA', 'TELEFONO', 'CELULAR', 'INTERNET', 'SERV_TV_PAGA',
    'SERV_PEL_PAGA', 'CON_VJUEGOS',
    'JEFE_SEXO',
    'INGTRHOG', 'TIPOHOG'
]]

print(f'We keep {pcat.shape[0]/personas.shape[0]*100}% and {vcat.shape[0]/viviendas.shape[0]*100} of the people and household datasets respectively.')

We keep 95.13226758864869% and 95.63444031938158 of the people and household datasets respectively.


/Users/gperaza/Drive/Research/population-synthesis/notebooks/../src/extended_survey.py:877: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  # 9: 'No especificado',
/Users/gperaza/Drive/Research/population-synthesis/notebooks/../src/extended_survey.py:892: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  'Tenía trabajo pero no trabajó': 'Trabaja',
/Users/gperaza/Drive/Research/population-synthesis/notebooks/../src/extended_survey.py:895: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the

In [17]:
from constraints import get_ind_const, get_viv_const

# Load constraints
constraints_ind = get_ind_const()
constraints_viv = get_viv_const()

print(f'We have a total of {len(constraints_ind)} people level constraints and {len(constraints_viv)} of household level constraints.')

We have a total of 58 people level constraints and 5 of household level constraints.


In [18]:
from census import process_census

# Load census
census_iter_path = Path('../data/census_loc/ITER_19CSV20.csv')
census_resageburb_path = Path('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv')
(
    df_mun, df_loc, df_agebs
) = process_census(census_iter_path, census_resageburb_path)

In [19]:
from misc import met_zone

In [20]:
# Build population matrices
from setup_lin_system import make_init_system, get_W

X, I, J, L, Up, Uh, U, Yp, Yh, Y, C = make_init_system(pcat, vcat, constraints_ind, constraints_viv, df_mun)

mun_list = X.MUN.unique()
const_zeroprob_list = []
for mun in mun_list:
    mun_mask = Y.MUN == mun
    U_mun = U.loc[:, mun_mask]
    const_zeroprob_list.extend(U_mun.index[U_mun.T.sum() == 0].to_list())
assert len(set(const_zeroprob_list)) == 0, set(const_zeroprob_list)

In [21]:
from taz import load_taz

# Import TAZ data
taz_path='taz_census.gpkg'
taz_dict = load_taz(taz_path, mun_list)

In [22]:
for df in taz_dict.values():
    df = df.dropna()
    assert np.all(df.POBTOT == df.POBHOG + df.POBCOL)
    assert np.all(df.TVIVHAB == df.TOTHOG + df.TOTCOL)    

In [23]:
C[["POBHOG", "P_0A2", "P_3A5", "P_6A11", "P_8A14", "P_12A14", "P_15A17", "P_18A24", "P_60YMAS", "P3A5_NOA", "P6A14NOA", "P15A17A", "P18A24A"]].head()

,POBHOG,P_0A2,P_3A5,P_6A11,P_8A14,P_12A14,P_15A17,P_18A24,P_60YMAS,P3A5_NOA,P6A14NOA,P15A17A,P18A24A
Abasolo,2974.0,165.0,166.0,314.0,368.0,152.0,168.0,358.0,328.0,52.0,34.0,115.0,96.0
Agualeguas,3376.0,114.0,141.0,253.0,319.0,152.0,152.0,259.0,878.0,34.0,16.0,100.0,52.0
Los Aldamas,1407.0,51.0,64.0,140.0,153.0,47.0,44.0,102.0,380.0,17.0,6.0,24.0,8.0
Allende,35095.0,1621.0,1934.0,3369.0,4000.0,1688.0,1753.0,4129.0,4687.0,814.0,480.0,1076.0,897.0
Anáhuac,18030.0,1012.0,1012.0,1931.0,2297.0,981.0,973.0,1850.0,2651.0,392.0,186.0,644.0,490.0


In [24]:
pop_l1 = pd.read_csv("../output/synth_people_L1.csv")
pop_l2 = pd.read_csv("../output/synth_people_L2.csv")
h_l1 = pd.read_csv("../output/synth_households_L1.csv")
h_l2 = pd.read_csv("../output/synth_households_L2.csv")

In [31]:
p_h_l1 = pd.merge(pop_l1, h_l1, on="HouseholdId", how="left")
p_h_l2 = pd.merge(pop_l2, h_l2, on="HouseholdId", how="left")

In [55]:
p_h_l1

,HouseholdId,PersonNumber,Age,Sex,License,TransitPass,EmploymentStatus,Occupation,FreeParking,StudentStatus,ExpansionFactor_x,HouseholdZone,ExpansionFactor_y,NumberOfPersons,DwellingType,Vehicles,IncomeClass
0,1,1,9.0,F,N,N,O,O,O,S,1,391,1,5,1,0,3
1,1,2,36.0,M,Y,N,P,M,O,O,1,391,1,5,1,0,3
2,1,3,1.0,M,N,N,O,O,O,O,1,391,1,5,1,0,3
3,1,4,6.0,F,N,N,O,O,O,O,1,391,1,5,1,0,3
4,1,5,30.0,F,Y,N,O,O,O,O,1,391,1,5,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040775,1193325,2,18.0,F,Y,N,P,S,O,O,1,577,1,3,1,1,4
4040776,1193325,3,46.0,F,Y,N,O,O,O,O,1,577,1,3,1,1,4
4040777,1193326,1,50.0,M,Y,N,P,P,O,O,1,579,1,3,1,1,4
4040778,1193326,2,18.0,F,Y,N,P,S,O,O,1,579,1,3,1,1,4


In [59]:
n_students_l1 = p_h_l1.query("StudentStatus == 'S' & Age <= 24").groupby("HouseholdZone").ExpansionFactor_x.sum()
n_students_l2 = p_h_l2.query("StudentStatus == 'S' & Age <= 24").groupby("HouseholdZone").ExpansionFactor_x.sum()

In [60]:
zone_list = []
for mun, df in taz_dict.items():
    zone_list.append(df.query("ZONA != -10"))

In [61]:
zone_gdf = pd.concat(zone_list).set_index("ZONA").sort_index()

In [65]:
total_s = zone_gdf[["P_3A5", "P_6A11", "P_12A14", "P3A5_NOA", "P6A11_NOA", "P12A14NOA", "P15A17A", "P18A24A"]].assign(
    P3A5A=lambda df: df.P_3A5 - df.P3A5_NOA,
    P6A11A=lambda df: df.P_6A11 - df.P6A11_NOA,
    P12A14A=lambda df: df.P_12A14 - df.P12A14NOA,
).drop(columns=["P_3A5", "P_6A11", "P_12A14", "P3A5_NOA", "P6A11_NOA", "P12A14NOA"]).sum(axis=1)

In [105]:
pd.concat([total_s, n_students_l1, n_students_l2], axis=1).dropna().sum()

0                    1284180.0
ExpansionFactor_x    1281838.0
ExpansionFactor_x    1275590.0
dtype: float64

In [122]:
pop_l1_c = pop_l1.query("3 <= Age <= 24").assign(AGE_C=pd.cut(pop_l1.query("3 <= Age <= 24").Age, bins=[2.5, 5.5, 11.5, 14.5, 17.5, 24.5]))
pop_l2_c = pop_l2.query("3 <= Age <= 24").assign(AGE_C=pd.cut(pop_l2.query("3 <= Age <= 24").Age, bins=[2.5, 5.5, 11.5, 14.5, 17.5, 24.5]))

In [140]:
l1_s = pd.crosstab(pop_l1_c.AGE_C, pop_l1_c.StudentStatus, values=pop_l1_c.ExpansionFactor, aggfunc='sum')[['S']].set_axis(
    ["P3A5A", "P6A11A", "P12A14A", "P15A17A", "P18A24A"]
).rename(columns={"S": "synth_new"})

In [141]:
l2_s = pd.crosstab(pop_l2_c.AGE_C, pop_l2_c.StudentStatus, values=pop_l2_c.ExpansionFactor, aggfunc='sum')[['S']].set_axis(
    ["P3A5A", "P6A11A", "P12A14A", "P15A17A", "P18A24A"]
).rename(columns={"S": "synth_old"})

In [134]:
zone_S = zone_gdf[["P_3A5", "P_6A11", "P_12A14", "P3A5_NOA", "P6A11_NOA", "P12A14NOA", "P15A17A", "P18A24A"]].assign(
    P3A5A=lambda df: df.P_3A5 - df.P3A5_NOA,
    P6A11A=lambda df: df.P_6A11 - df.P6A11_NOA,
    P12A14A=lambda df: df.P_12A14 - df.P12A14NOA,
).drop(columns=["P_3A5", "P_6A11", "P_12A14", "P3A5_NOA", "P6A11_NOA", "P12A14NOA"]).sum().loc[["P3A5A", "P6A11A", "P12A14A", "P15A17A", "P18A24A"]]
zone_S

P3A5A      168085.0
P6A11A     483812.0
P12A14A    233642.0
P15A17A    181374.0
P18A24A    217267.0
dtype: float64

In [146]:
zone_S

P3A5A      168085.0
P6A11A     483812.0
P12A14A    233642.0
P15A17A    181374.0
P18A24A    217267.0
Name: Total, dtype: float64

In [167]:
student_counts = pd.concat([zone_S.rename('Census'), l1_s, l2_s], axis=1).assign(
    diff_old = lambda df: df.Census - df.synth_old,
    diff_new = lambda df: df.Census - df.synth_new,
    diff_old_abs = lambda df: abs(df.Census - df.synth_old),
    diff_new_abs = lambda df: abs(df.Census - df.synth_new),
    frac_diff_old = lambda df: abs(df.Census - df.synth_old)/df.Census*100,
    frac_diff_new = lambda df: abs(df.Census - df.synth_new)/df.Census*100,
)
student_counts.loc["totals"] = student_counts.sum()
student_counts

,Census,synth_new,synth_old,diff_old,diff_new,diff_old_abs,diff_new_abs,frac_diff_old,frac_diff_new
P3A5A,168085.0,168165.0,162269.0,5816.0,-80.0,5816.0,80.0,3.460154,0.047595
P6A11A,483812.0,483802.0,478490.0,5322.0,10.0,5322.0,10.0,1.100014,0.002067
P12A14A,233642.0,233465.0,229981.0,3661.0,177.0,3661.0,177.0,1.566927,0.075757
P15A17A,181374.0,179975.0,182911.0,-1537.0,1399.0,1537.0,1399.0,0.847420,0.771334
P18A24A,217267.0,216431.0,221941.0,-4674.0,836.0,4674.0,836.0,2.151270,0.384780
totals,1284180.0,1281838.0,1275592.0,8588.0,2342.0,21010.0,2502.0,9.125786,1.281533


In [154]:
student_counts.diff_new.sum()

2342.0

In [157]:
student_counts.sum()

Census       1284180.0
synth_new    1281838.0
synth_old    1275592.0
diff_old        8588.0
diff_new        2342.0
dtype: float64

In [169]:
od_p = pd.read_csv("../output/od_people.csv")

In [172]:
od_p_c = od_p.query("3 <= Age <= 24").assign(AGE_C=pd.cut(od_p.query("3 <= Age <= 24").Age, bins=[2.5, 5.5, 11.5, 14.5, 17.5, 24.5]))

In [181]:
od_scounts = pd.crosstab(od_p_c.AGE_C, od_p_c.StudentStatus, values=od_p_c.ExpansionFactor, aggfunc='sum')[['S']].set_axis(
    ["P3A5A", "P6A11A", "P12A14A", "P15A17A", "P18A24A"]
).rename(columns={"S": "OD"})
od_scounts.loc["totals"] = od_scounts.sum()
od_scounts

StudentStatus,OD
P3A5A,0.000000e+00
P6A11A,3.817474e+05
P12A14A,2.072441e+05
P15A17A,1.985347e+05
P18A24A,2.670151e+05
totals,1.054541e+06


In [191]:
pd.concat([student_counts.Census, od_scounts], axis=1).astype(int).assign(diff=lambda df: df.Census - df.OD)

,Census,OD,diff
P3A5A,168085,0,168085
P6A11A,483812,381747,102065
P12A14A,233642,207244,26398
P15A17A,181374,198534,-17160
P18A24A,217267,267015,-49748
totals,1284180,1054541,229639


In [ ]:
Census	synth_new	synth_old	diff_old	diff_new	diff_old_abs	diff_new_abs	frac_diff_old	frac_diff_new
P3A5A	168085.0	168165.0	162269.0	5816.0	-80.0	5816.0	80.0	3.460154	0.047595
P6A11A	483812.0	483802.0	478490.0	5322.0	10.0	5322.0	10.0	1.100014	0.002067
P12A14A	233642.0	233465.0	229981.0	3661.0	177.0	3661.0	177.0	1.566927	0.075757
P15A17A	181374.0	179975.0	182911.0	-1537.0	1399.0	1537.0	1399.0	0.847420	0.771334
P18A24A	217267.0	216431.0	221941.0	-4674.0	836.0	4674.0	836.0	2.151270	0.384780
totals	1284180.0	1281838.0	1275592.0	8588.0	2342.0	21010.0	2502.0	9.125786	1.281533
